# **GAI Project 3 PEFT on GLUE benchmarks - lora**

## 環境設置

In [59]:
# 若沒有安裝 transformers 和 datasets 套件，請取消以下註解並執行
! pip install transformers
! pip install datasets
! pip install torch
! pip install peft
! pip install evaluate
! pip install transformers[torch]

In [60]:
# 1. 確認所需套件的版本
import torch
print("PyTorch 的版本為: {}".format(torch.__version__))

import transformers
print("Hugging Face Transformers 的版本為: {}".format(transformers.__version__))

import datasets
print("Hugging Face Datasets 的版本為: {}".format(datasets.__version__))

import peft
print("PEFT 的版本為: {}".format(peft.__version__))

PyTorch 的版本為: 2.2.1+cu121
Hugging Face Transformers 的版本為: 4.40.2
Hugging Face Datasets 的版本為: 2.19.1
PEFT 的版本為: 0.10.0


In [61]:
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    LoraConfig,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
    TaskType,
)

In [62]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import re
import warnings
import json
import os
import evaluate
import numpy as np
from pathlib import Path # (Python3.4+)
device = "cuda" if torch.cuda.is_available() else "cpu"
warnings.filterwarnings("ignore")

## 加入tokenizer

In [63]:
checkpoint = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [64]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## **SST2**

### 資料處理

In [65]:
data_sst = load_dataset("GLUE", "sst2")
data_sst

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [66]:
dataset_ratio = 0.1

# 切割資料
train_size = int(len(data_sst["train"])*dataset_ratio)
validation_size = int(len(data_sst["validation"])*dataset_ratio)
test_size = int(len(data_sst["test"])*dataset_ratio)

data_sst["train"] = data_sst["train"].select(range(train_size))
data_sst["validation"] = data_sst["validation"].select(range(validation_size))
data_sst["test"] = data_sst["test"].select(range(test_size))

data_sst

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 6734
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 87
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 182
    })
})

In [67]:
def preprocess_function(batch):
  return tokenizer(batch["sentence"], truncation=True)

In [68]:
sst2_tokenized_datasets = data_sst.map(preprocess_function, batched=True)
sst2_tokenized_datasets

Map:   0%|          | 0/87 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6734
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 87
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 182
    })
})

### 引入模型

In [69]:
sst2_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### PEFT(lora) and Evaluate

In [70]:
def sst2_compute_metrics(eval_preds):
    metric = evaluate.load("glue", "sst2")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [71]:
peft_config = LoraConfig(
        r=8,         # Lora r
        lora_alpha=8,    # Lora alpha
        lora_dropout=0.01,  # Lora dropout
        bias="none",
        task_type="SEQ_CLS",
)

sst2_model = get_peft_model(sst2_model, peft_config)
sst2_model.print_trainable_parameters()

trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700395193203643


### Train

In [72]:
# 訓練模型

# 設定 TrainingArguments
sst2_training_args = TrainingArguments(
    output_dir="./results",          # 輸出的資料夾
    num_train_epochs=5,            # 總共訓練的 epoch 數目
    learning_rate=5e-4,            # 學習率
    per_device_train_batch_size=16,      # 訓練模型時每個裝置的 batch size
    per_device_eval_batch_size=16,      # 驗證模型時每個裝置的 batch size
    gradient_accumulation_steps=1,      # 梯度累積的步數
    warmup_steps=500,             # learning rate scheduler 的參數
    weight_decay=0.01,            # 最佳化演算法 (optimizer) 中的權重衰退率
    evaluation_strategy="epoch",       # 設定驗證的時機
    save_strategy="epoch",          # 設定儲存的時機
    save_total_limit=10,           # 最多儲存幾個模型
    logging_dir='./logs',           # 存放 log 的資料夾
    logging_steps=150,
    seed= 42,
)

In [73]:
sst2_trainer = Trainer(
    model=sst2_model,                     # 🤗 的模型
    args=sst2_training_args,                  #  Trainer 所需要的引數
    train_dataset=sst2_tokenized_datasets["train"],      # 訓練集 (注意是 PyTorch Dataset)
    eval_dataset=sst2_tokenized_datasets["validation"],    # 驗證集 (注意是 PyTorch Dataset)，可使 Trainer 在進行訓練時也進行驗證
    tokenizer=tokenizer,
    compute_metrics=sst2_compute_metrics,           # 自定的評估的指標
)

In [74]:
# 指定使用 1 個 GPU 進行訓練
sst2_trainer.args._n_gpu=1

# 開始進行模型訓練
sst2_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.470200,0.241471,0.896552
2,0.277600,0.228126,0.896552
3,0.216800,0.315291,0.896552
4,0.175600,0.310487,0.908046
5,0.152700,0.312068,0.896552


TrainOutput(global_step=2105, training_loss=0.2764841381416185, metrics={'train_runtime': 176.7314, 'train_samples_per_second': 190.515, 'train_steps_per_second': 11.911, 'total_flos': 607735783214160.0, 'train_loss': 0.2764841381416185, 'epoch': 5.0})

# CoLA

### 資料處理

In [75]:
data_cola = load_dataset("GLUE", "cola")
data_cola

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [76]:
def cola_preprocess_function(batch):
  return tokenizer(batch["sentence"], truncation=True)

In [77]:
cola_tokenized_datasets = data_cola.map(cola_preprocess_function, batched=True)
cola_tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1063
    })
})

### 引入模型

In [78]:
cola_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### PEFT(lora) and Evaluate

In [79]:
def cola_compute_metrics(eval_preds):
    metric = evaluate.load("glue", "cola")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [80]:
peft_config = LoraConfig(
        r=8,         # Lora r
        lora_alpha=8,    # Lora alpha
        lora_dropout=0.01,  # Lora dropout
        bias="none",
        task_type="SEQ_CLS",
)

cola_model = get_peft_model(cola_model, peft_config)
sst2_model.print_trainable_parameters()

trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700395193203643


### train

In [81]:
# 訓練模型

# 設定 TrainingArguments
cola_training_args = TrainingArguments(
    output_dir="./results",          # 輸出的資料夾
    num_train_epochs=5,            # 總共訓練的 epoch 數目
    learning_rate=1e-3,            # 學習率
    per_device_train_batch_size=16,      # 訓練模型時每個裝置的 batch size
    per_device_eval_batch_size=16,      # 驗證模型時每個裝置的 batch size
    gradient_accumulation_steps=1,      # 梯度累積的步數
    warmup_steps=500,             # learning rate scheduler 的參數
    weight_decay=0.01,            # 最佳化演算法 (optimizer) 中的權重衰退率
    evaluation_strategy="epoch",       # 設定驗證的時機
    save_strategy="epoch",          # 設定儲存的時機
    save_total_limit=10,           # 最多儲存幾個模型
    logging_dir='./logs',           # 存放 log 的資料夾
    logging_steps=150,
    seed= 42,
)

In [82]:
cola_trainer = Trainer(
    model=cola_model,                         # 🤗 的模型
    args=cola_training_args,                  # Trainer 所需要的引數
    train_dataset=cola_tokenized_datasets["train"],          # 訓練集 (注意是 PyTorch Dataset)
    eval_dataset=cola_tokenized_datasets["validation"],      # 驗證集 (注意是 PyTorch Dataset)，可使 Trainer 在進行訓練時也進行驗證
    tokenizer=tokenizer,
    compute_metrics=cola_compute_metrics,     # 自定的評估的指標
)

In [83]:
# 指定使用 1 個 GPU 進行訓練
cola_trainer.args._n_gpu=1

# 開始進行模型訓練
cola_trainer.train()

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.489100,0.427919,0.538214
2,0.425500,0.430109,0.525929
3,0.350100,0.436294,0.557342
4,0.279300,0.625019,0.543489
5,0.215600,0.526561,0.577918


TrainOutput(global_step=2675, training_loss=0.37090666726370836, metrics={'train_runtime': 157.9165, 'train_samples_per_second': 270.744, 'train_steps_per_second': 16.939, 'total_flos': 456422958890280.0, 'train_loss': 0.37090666726370836, 'epoch': 5.0})

# MRPC

### 資料處理

In [84]:
data = load_dataset("GLUE", "mrpc")
data

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [85]:
mrpc_tokenized_datasets = data.map(lambda x: tokenizer(x["sentence1"], x["sentence2"]), batched=True)
mrpc_tokenized_datasets

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

### 引入模型

In [86]:
mrpc_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### PEFT(lora) and Evaluate

In [87]:
def mrpc_compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [88]:
peft_config = LoraConfig(
        r=8,         # Lora r
        lora_alpha=8,    # Lora alpha
        lora_dropout=0.01,  # Lora dropout
        bias="none",
        task_type="SEQ_CLS",
)

mrpc_model = get_peft_model(mrpc_model, peft_config)
sst2_model.print_trainable_parameters()

trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700395193203643


### train

In [89]:
# 訓練模型

# 設定 TrainingArguments
mrpc_training_args = TrainingArguments(
    output_dir="./results",          # 輸出的資料夾
    num_train_epochs=5,            # 總共訓練的 epoch 數目
    learning_rate=5e-4,            # 學習率
    per_device_train_batch_size=16,      # 訓練模型時每個裝置的 batch size
    per_device_eval_batch_size=16,      # 驗證模型時每個裝置的 batch size
    gradient_accumulation_steps=1,      # 梯度累積的步數
    warmup_steps=500,             # learning rate scheduler 的參數
    weight_decay=0.01,            # 最佳化演算法 (optimizer) 中的權重衰退率
    evaluation_strategy="epoch",       # 設定驗證的時機
    save_strategy="epoch",          # 設定儲存的時機
    save_total_limit=10,           # 最多儲存幾個模型
    logging_dir='./logs',           # 存放 log 的資料夾
    logging_steps=150,
    seed= 42,
)

In [90]:
mrpc_trainer = Trainer(
    model=mrpc_model,                         # 🤗 的模型
    args=mrpc_training_args,                  # Trainer 所需要的引數
    train_dataset=mrpc_tokenized_datasets["train"],          # 訓練集 (注意是 PyTorch Dataset)
    eval_dataset=mrpc_tokenized_datasets["validation"],      # 驗證集 (注意是 PyTorch Dataset)，可使 Trainer 在進行訓練時也進行驗證
    tokenizer=tokenizer,
    compute_metrics=mrpc_compute_metrics,     # 自定的評估的指標
)

In [91]:
# 指定使用 1 個 GPU 進行訓練
mrpc_trainer.args._n_gpu=1

# 開始進行模型訓練
mrpc_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.647600,0.550347,0.737745,0.830964
2,0.502800,0.397069,0.830882,0.881239
3,0.414700,0.353348,0.850490,0.888073
4,0.316800,0.330763,0.867647,0.905594
5,0.270800,0.354508,0.875000,0.910369


TrainOutput(global_step=1150, training_loss=0.42156298098356826, metrics={'train_runtime': 189.3295, 'train_samples_per_second': 96.868, 'train_steps_per_second': 6.074, 'total_flos': 717425475554880.0, 'train_loss': 0.42156298098356826, 'epoch': 5.0})